# Datorseenede projekt 
#### Projektet i kursen datorseende

In [1]:
%pip cache purge

Files removed: 0 (0 bytes)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip uninstall torchvision -y


Found existing installation: torchvision 0.21.0+cu126
Uninstalling torchvision-0.21.0+cu126:
  Successfully uninstalled torchvision-0.21.0+cu126
Note: you may need to restart the kernel to use updated packages.


In [1]:

%pip install torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.21.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
Using cached https://download.pytorch.org/whl/cu126/torchvision-0.21.0%2Bcu126-cp311-cp311-win_amd64.whl (6.1 MB)
Note: you may need to restart the kernel to use updated packages.


### Installationer
Installerar paket som behövs för exekveringen av koden.

In [4]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


### Spjälka data 
Spjälker data för att kunna användas i yolo modellen. Data spjälks till test, train och val.

In [7]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.7,0.2,0.1),group_prefix=None)

Copying files: 5302 files [00:24, 212.72 files/s]


### Testa installation
för att kolla att allt är installerat korrekt och att versionerna matchar

In [1]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


## Utför träningen 
Träna en yolov11 model med datan som vi har, med passande parametrar för egen dator att köra.

In [ ]:
from ultralytics import YOLO


# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")


results = model.train(
    data="config.yaml",         # Din träningsdata
    project="./output",         # Utmatningsmapp
    device='cuda:0',            # Träna på GPU (ändra om du vill använda CPU)
    epochs=10,                  # Antal epoker
    batch=16,                   # Batch size, hur många bilder åt gången
    imgsz=640,                  # Bildstorlek, 640x640 bör vara bra för din GPU
    amp=True,                   # Aktivera Automatic Mixed Precision för snabbare träning och mindre minne
    optimizer='SGD',            # Använd Adam optimizer för bättre konvergens
    workers=8,                 # Antal arbetare för dataladdning, öka om du har fler CPU-kärnor       
    name="yolo_model_test",          # Namn på träningssessionen
    patience=50,                # Stoppar om ingen förbättring efter 50 epochs
    exist_ok=True,              # Om du vill skriva över tidigare resultat
    lr0=0.001,                  # Learning rate schemaläggare
    lrf=0.2
)
# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
success = model.export(format="onnx")

Ultralytics 8.3.89  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=config.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=12, project=./output, name=yolo_model_test, exist_ok=True, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning C:\Projects\VisionComp\cv-25-project\dataset\train\labels.cache... 1855 images, 16 backgrounds, 0 corrupt: 100%|██████████| 1855/1855 [00:00<?, ?it/s]
val: Scanning C:\Projects\VisionComp\cv-25-project\dataset\val\labels.cache... 530 images, 5 backgrounds, 0 corrupt: 100%|██████████| 530/530 [00:00<?, ?it/s]


KeyboardInterrupt: 